# financial statements testing grounds


In [1]:
import yfinance as yf
import pandas as pd

In [97]:
import requests
import time
api_key = "TULH49F5XQMU8MQ5"
base_url = "https://www.alphavantage.co/query"

def get_fin_stmt(symbol, function):
    time.sleep(60)
    url = base_url + "?function=" + function + "&symbol=" + symbol + "&apikey=" + api_key
    response = requests.get(url)
    if response.status_code != 200:
        print("Error, cannot retrieve" + function + "for " + symbol)
        return None
    data = response.json()
    if "annualReports" not in data:
        print("Error: No " + function + " data found for " + symbol)
        return None
#     print(data)
    statement_data = data["annualReports"]
    return_dct = {}
    
    return statement_data
statement_dct = {}

statements = {}
stocks = ["ENPH", "IBM"]

statement_types =  ["INCOME_STATEMENT", "BALANCE_SHEET", "CASH_FLOW"]
for stock in stocks:
    statements[stock] = {}
    for statement_type in statement_types:
        statement_key = statement_type.split("_")[0]
        statements[stock][statement_key] = {}
        try:
            fin_statement = get_fin_stmt(stock, statement_type)
            for annual_data in fin_statement:
                date = annual_data["fiscalDateEnding"]

                statements[stock][statement_key][date] = annual_data
            statements[stock][statement_key] = pd.DataFrame(income_statements[stock])
        except:
            print("Error retrieving " + statement_type + "for " + stock)
income_statements

Error retrieving INCOME_STATEMENTfor IBM
Error retrieving BALANCE_SHEETfor IBM
Error retrieving CASH_FLOWfor IBM


{'ENPH':                                    2022-12-31  2021-12-31  2020-12-31  \
 fiscalDateEnding                   2022-12-31  2021-12-31  2020-12-31   
 reportedCurrency                          USD         USD         USD   
 grossProfit                         974595000   554422000   345981000   
 totalRevenue                       2330853000  1382049000   774425000   
 costOfRevenue                      1356258000   827627000   428444000   
 costofGoodsAndServicesSold         1356258000   827627000   428444000   
 operatingIncome                     448261000   215832000   186439000   
 sellingGeneralAndAdministrative     355104000   233064000   103621000   
 researchAndDevelopment              168846000   105526000    55921000   
 operatingExpenses                   526334000   338590000   159542000   
 investmentIncomeNet                      None        None        None   
 netInterestIncome                     4218000   -44457000   -18845000   
 interestIncome               

In [94]:
statements

{'ENPH': {'INCOME':                                    2022-12-31  2021-12-31  2020-12-31  \
  fiscalDateEnding                   2022-12-31  2021-12-31  2020-12-31   
  reportedCurrency                          USD         USD         USD   
  grossProfit                         974595000   554422000   345981000   
  totalRevenue                       2330853000  1382049000   774425000   
  costOfRevenue                      1356258000   827627000   428444000   
  costofGoodsAndServicesSold         1356258000   827627000   428444000   
  operatingIncome                     448261000   215832000   186439000   
  sellingGeneralAndAdministrative     355104000   233064000   103621000   
  researchAndDevelopment              168846000   105526000    55921000   
  operatingExpenses                   526334000   338590000   159542000   
  investmentIncomeNet                      None        None        None   
  netInterestIncome                     4218000   -44457000   -18845000   
  inter

In [70]:
fin_statement[0]#["fiscalDateEnding"]

{'fiscalDateEnding': '2022-12-31',
 'reportedCurrency': 'USD',
 'grossProfit': '974595000',
 'totalRevenue': '2330853000',
 'costOfRevenue': '1356258000',
 'costofGoodsAndServicesSold': '1356258000',
 'operatingIncome': '448261000',
 'sellingGeneralAndAdministrative': '355104000',
 'researchAndDevelopment': '168846000',
 'operatingExpenses': '526334000',
 'investmentIncomeNet': 'None',
 'netInterestIncome': '4218000',
 'interestIncome': '13656000',
 'interestExpense': '9438000',
 'nonInterestIncome': '2330853000',
 'otherNonOperatingIncome': '-431000',
 'depreciation': '27700000',
 'depreciationAndAmortization': '24696000',
 'incomeBeforeTax': '452048000',
 'incomeTaxExpense': '54686000',
 'interestAndDebtExpense': '9438000',
 'netIncomeFromContinuingOperations': '397362000',
 'comprehensiveIncomeNetOfTax': '388500000',
 'ebit': '461486000',
 'ebitda': '486182000',
 'netIncome': '397362000'}

In [54]:
ENPH_Over = get_fin_stmt("IBM", " OVERVIEW")
ENPH_Over

Error: No  OVERVIEW data found for IBM


In [58]:
data_dct = {}
for json_dct in ENPH_IS:
#     date = json_dct["fiscalDateEnding"]
    data_dct[date] = json_dct

pd.DataFrame(data_dct).T

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome
2022-12-31,2022-12-31,USD,974595000,2330853000,1356258000,1356258000,448261000,355104000,168846000,526334000,...,27700000,24696000,452048000,54686000,9438000,397362000,388500000,461486000,486182000,397362000
2021-12-31,2021-12-31,USD,554422000,1382049000,827627000,827627000,215832000,233064000,105526000,338590000,...,16700000,9500000,120928000,-24521000,101649000,145449000,142995000,166080000,175580000,145449000
2020-12-31,2020-12-31,USD,345981000,774425000,428444000,428444000,186439000,103621000,55921000,159542000,...,9700000,5092000,119410000,-14585000,21001000,133995000,135352000,140411000,145503000,133995000
2019-12-31,2019-12-31,USD,221245000,624333000,403088000,403088000,102729000,75536000,40381000,118516000,...,7300000,4727000,90114000,-71034000,17843000,161148000,159483000,99805000,104532000,161148000
2018-12-31,2018-12-31,USD,94445000,316159000,221714000,221714000,1596000,56133000,32587000,92849000,...,8300000,1400000,-10229000,1398000,9635000,-11627000,-10229000,464000,1864000,-11627000


In [35]:
for var in ENPH_IS:
    newENPH_IS = var
newENPH_IS

{'fiscalDateEnding': '2018-12-31',
 'reportedCurrency': 'USD',
 'grossProfit': '94445000',
 'totalRevenue': '316159000',
 'costOfRevenue': '221714000',
 'costofGoodsAndServicesSold': '221714000',
 'operatingIncome': '1596000',
 'sellingGeneralAndAdministrative': '56133000',
 'researchAndDevelopment': '32587000',
 'operatingExpenses': '92849000',
 'investmentIncomeNet': 'None',
 'netInterestIncome': '-9635000',
 'interestIncome': '1058000',
 'interestExpense': '10693000',
 'nonInterestIncome': '316159000',
 'otherNonOperatingIncome': '-2190000',
 'depreciation': '8300000',
 'depreciationAndAmortization': '1400000',
 'incomeBeforeTax': '-10229000',
 'incomeTaxExpense': '1398000',
 'interestAndDebtExpense': '9635000',
 'netIncomeFromContinuingOperations': '-11627000',
 'comprehensiveIncomeNetOfTax': '-10229000',
 'ebit': '464000',
 'ebitda': '1864000',
 'netIncome': '-11627000'}

In [44]:
newENPH_IS.keys()
for i in newENPH_IS:
    print(i, newENPH_IS[i])

fiscalDateEnding 2018-12-31
reportedCurrency USD
grossProfit 94445000
totalRevenue 316159000
costOfRevenue 221714000
costofGoodsAndServicesSold 221714000
operatingIncome 1596000
sellingGeneralAndAdministrative 56133000
researchAndDevelopment 32587000
operatingExpenses 92849000
investmentIncomeNet None
netInterestIncome -9635000
interestIncome 1058000
interestExpense 10693000
nonInterestIncome 316159000
otherNonOperatingIncome -2190000
depreciation 8300000
depreciationAndAmortization 1400000
incomeBeforeTax -10229000
incomeTaxExpense 1398000
interestAndDebtExpense 9635000
netIncomeFromContinuingOperations -11627000
comprehensiveIncomeNetOfTax -10229000
ebit 464000
ebitda 1864000
netIncome -11627000


In [32]:
test_data = {}
for var in ENPH_IS:
    print(var)
    

{'fiscalDateEnding': '2022-12-31', 'reportedCurrency': 'USD', 'grossProfit': '974595000', 'totalRevenue': '2330853000', 'costOfRevenue': '1356258000', 'costofGoodsAndServicesSold': '1356258000', 'operatingIncome': '448261000', 'sellingGeneralAndAdministrative': '355104000', 'researchAndDevelopment': '168846000', 'operatingExpenses': '526334000', 'investmentIncomeNet': 'None', 'netInterestIncome': '4218000', 'interestIncome': '13656000', 'interestExpense': '9438000', 'nonInterestIncome': '2330853000', 'otherNonOperatingIncome': '-431000', 'depreciation': '27700000', 'depreciationAndAmortization': '24696000', 'incomeBeforeTax': '452048000', 'incomeTaxExpense': '54686000', 'interestAndDebtExpense': '9438000', 'netIncomeFromContinuingOperations': '397362000', 'comprehensiveIncomeNetOfTax': '388500000', 'ebit': '461486000', 'ebitda': '486182000', 'netIncome': '397362000'}
{'fiscalDateEnding': '2021-12-31', 'reportedCurrency': 'USD', 'grossProfit': '554422000', 'totalRevenue': '1382049000', 

In [12]:
stocks = ["AAPL", "ENPH", "MSFT"]
data = {}
for stock in stocks:
    data[stock] = {}
    data[stock]["Income Statement"] = get_fin_stmt(symbol = stock, stmt_type = "INCOME_STATEMENT")
    data[stock]["Balance Sheet"] = get_fin_stmt(symbol = stock, stmt_type = "BALANCE_SHEET")
    data[stock]["Cash Flow"] = get_fin_stmt(symbol = stock, stmt_type = "CASH_FLOW")

Error: No BALANCE_SHEET data found forENPH
Error: No CASH_FLOW data found forENPH
Error: No INCOME_STATEMENT data found forMSFT
Error: No BALANCE_SHEET data found forMSFT
Error: No CASH_FLOW data found forMSFT


In [59]:
data

{'AAPL': {'Income Statement': [{'fiscalDateEnding': '2022-09-30',
    'reportedCurrency': 'USD',
    'grossProfit': '170782000000',
    'totalRevenue': '391397000000',
    'costOfRevenue': '248640000000',
    'costofGoodsAndServicesSold': '223546000000',
    'operatingIncome': '119437000000',
    'sellingGeneralAndAdministrative': '25094000000',
    'researchAndDevelopment': '26251000000',
    'operatingExpenses': '51345000000',
    'investmentIncomeNet': '2825000000',
    'netInterestIncome': '-2931000000',
    'interestIncome': '106000000',
    'interestExpense': '2931000000',
    'nonInterestIncome': '394328000000',
    'otherNonOperatingIncome': '-228000000',
    'depreciation': '8700000000',
    'depreciationAndAmortization': '11104000000',
    'incomeBeforeTax': '119103000000',
    'incomeTaxExpense': '19300000000',
    'interestAndDebtExpense': '2931000000',
    'netIncomeFromContinuingOperations': '99803000000',
    'comprehensiveIncomeNetOfTax': '88531000000',
    'ebit': '122

In [5]:
type(data)

dict

import requests

# API key for Alpha Vantage
api_key = 'YOUR_API_KEY'

# Base URL for Alpha Vantage API
base_url = 'https://www.alphavantage.co/query'

# Ticker symbol for AAPL
symbol = 'AAPL'

# Initialize the data dictionary
data = {}

# Function to retrieve financial statement data for a given symbol and statement type
def get_financial_statement(symbol, statement_type):
    # Build the URL for the API call
    url = base_url + '?function=' + statement_type + '&symbol=' + symbol + '&apikey=' + api_key

    # Make the API call
    response = requests.get(url)

    # Check if the API call was successful
    if response.status_code != 200:
        print("Error: Unable to retrieve " + statement_type + " for " + symbol)
        return None

    # Extract the financial statement data from the API response
    data = response.json()
    statement_data = data['annualReports']

    return statement_data

# Retrieve and store the income statement for AAPL
income_statement = get_financial_statement(symbol, 'INCOME_STATEMENT')
data[symbol] = {'Income Statement': income_statement}

# Retrieve and store the balance sheet for AAPL
balance_sheet = get_financial_statement(symbol, 'BALANCE_SHEET')
data[symbol]['Balance Sheet'] = balance_sheet

# Retrieve and store the cash flow statement for AAPL
cash_flow = get_financial_statement(symbol, 'CASH_FLOW')
data[symbol]['Cash Flow'] = cash_flow

# Print the retrieved data
print(data[symbol]['Income Statement'])
print(data[symbol]['Balance Sheet'])
print(data[symbol]['Cash Flow'])
